## README
mlflow models can be served from the runs or from the registered models using the below methods:

<code>mlflow models serve --model-uri runs:/YOUR_MODEL_RUN/YOUR_MODEL_NAME -p PORT</code><br>
<code>mlflow models serve --model-uri models:/REGISTERED_MODEL_NAME/MODEL_VERSION -p PORT</code><br>
<code>mlflow models serve -m file:/<PATH_TO_MODEL> -p PORT </code>

The meta information for serving the model is retrieved from the location specified by the <code>MLFLOW_TRACKING_URI</code> environment variable

Refer : https://www.mlflow.org/docs/latest/model-registry.html

In [5]:
import pandas as pd
import numpy as np
import subprocess
import json
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import mlflow
import mlflow.sklearn

from load_data import DataLoader

In [2]:
dl = DataLoader()
dl.load_data('dataset/creditcard.csv', ['Class'], ['Time'])

In [57]:
batch_size = 80
num_batches = 30
result = []
for n in range(num_batches):
    input_json = pd.DataFrame(data=dl.x_test[n * batch_size:(n+1) * batch_size], 
                              columns=dl.featureColumns).to_json(orient="split")
    proc = subprocess.run(["curl",  "-X", "POST", "-H", 
                           "Content-Type:application/json; format=pandas-split", 
                           "--data", input_json, "http://127.0.0.1:31235/invocations"], 
                          stdout=subprocess.PIPE, encoding='utf-8')
    output = proc.stdout
    result = result+ json.loads(output)

In [58]:
df2 = pd.DataFrame(result)
print(classification_report(dl.y_test[:num_batches * batch_size], np.array(result).reshape(-1,1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2398
           1       0.67      1.00      0.80         2

    accuracy                           1.00      2400
   macro avg       0.83      1.00      0.90      2400
weighted avg       1.00      1.00      1.00      2400

